**Eular Method**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def EularhMethod(a,b,N,alpha,f):
  h=(b-a)/N;
  t=a
  w=alpha
  ts=np.zeros(N+1)
  ws=np.zeros(N+1)
  ts[0]=a
  ws[0]=alpha
  for i in range(1,N+1):
    w=w+h*f(t,w)
    t=a+i*h
    ts[i]=t
    ws[i]=w
  return ws,ts
def f(t,y):
  return y-t*t+1
def y(t):
  return (t+1)**2-0.5*np.exp(t)
alpha=0.5
a=0
b=2
N=51
ws,ts=EularhMethod(a,b,N,alpha,f)
plt.figure(1)
plt.plot(ts,ws,'o')
plt.plot(ts,y(ts),'b')
error=np.abs(ws-y(ts))
plt.figure(2)
plt.plot(ts,error)

**Runge-Kutta (Order Four)**

In [ ]:
import matplotlib.pyplot as plt
def RKM(a,b,N,alpha,f):
  h=(b-a)/N
  t=[]
  w=[]
  w.append(alpha)
  t.append(a)
  for i in range(0,N):
    K1=h*f(t[i],w[i])
    K2=h*f(t[i]+h/2,w[i]+K1/2)
    K3=h*f(t[i]+h/2,w[i]+K2/2)
    K4=h*f(t[i]+h,w[i]+K3)
    w.append(w[i]+(K1+2*K2+2*K3+K4)/6)
    t.append(a+(i+1)*h)
  return w,t
def f(t,y):
  return y-t*t+1
#def y(t):
 # return (t+1)**2-0.5*np.exp(t)
alpha=0.5
a=0
b=2
N=10
ws,ts=RKM(a,b,N,alpha,f)
plt.plot(ts,ws,'o')
#plt.plot(ts,y(ts),'b')

**Runge-Kutta Method for Systems of Differential Equations**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def system_of_equations(t, Y):
    # Define the system of differential equations
    # This is an example system with two equations
    dy1dt = -0.5 * Y[0]
    dy2dt = 0.5 * Y[0] - 0.2 * Y[1]
    return np.array([dy1dt, dy2dt])

def runge_kutta_4th_order_system(f, t0, y0, h, num_steps):
    # Fourth-order Runge-Kutta method for a system of ODEs
    t_values = [t0 + i * h for i in range(num_steps + 1)]
    y_values = np.zeros((num_steps + 1, len(y0)))
    y_values[0] = y0

    for i in range(num_steps):
        k1 = h * f(t_values[i], y_values[i])
        k2 = h * f(t_values[i] + 0.5 * h, y_values[i] + 0.5 * k1)
        k3 = h * f(t_values[i] + 0.5 * h, y_values[i] + 0.5 * k2)
        k4 = h * f(t_values[i] + h, y_values[i] + k3)

        y_values[i + 1] = y_values[i] + (k1 + 2*k2 + 2*k3 + k4) / 6

    return t_values, y_values

# Example usage:
t0 = 0
y0 = np.array([1.0, 0.0])  # Initial conditions for the system
h = 0.1  # Step size
num_steps = 100  # Number of steps

t_values, y_values = runge_kutta_4th_order_system(system_of_equations, t0, y0, h, num_steps)

# Plot the results
plt.plot(t_values, y_values[:, 0], label='y1(t)')
plt.plot(t_values, y_values[:, 1], label='y2(t)')
plt.xlabel('t')
plt.ylabel('y(t)')
plt.legend()
plt.show()


**Gaussian Elimination 2.0**

In [ ]:
#Author: Walter Nam
import sys
import pprint
import numpy as np
import scipy as sp
import sympy
from scipy import linalg


class Data:

    def __init__(self, A, b):
        self.A = A
        self.b = b

    def computed_sol(self, A, b):
        print("Numpy solution")
        try:
            return np.linalg.solve(A,b)
        except np.linalg.LinAlgError:
            print("No solution")
            return None

    def gaussian(self, A, b):
        n = len(b)
        x = np.zeros(n, float)
        for i in range (n-1):
            if np.fabs(A[i,i]) < sys.maxsize:
                for j in range(i+1, n):
                    if np.fabs(A[j,i]) > np.fabs(A[i,i]):
                        A[[i,j]] = A[[j,i]]
                        b[[i,j]] = b[[j,i]]
                        break
            for k in range (i+1, n):
                if A[k,i] == 0: continue
                factor = A[i,i] / A[k,i]
                for j in range(i,n):
                    A[k,j] = A[i,j] - A[k,j] * factor
                b[k] = b[i] - b[k] * factor

        #Backward substitution
        x[n - 1] = b[n - 1] / A[n - 1, n - 1]
        for i in range(n - 2, -1, -1):
            sum_ax = 0
            for j in range(i + 1, n):
                sum_ax += A[i, j] * x[j]
            x[i] = (b[i] - sum_ax) / A[i,i]

        return A, b, x

    def check_valid_solution(self, A, b, gauss, free_variable_solution):
        solution = gauss(A, b)[2]
        b_0 = gauss(A, b)[1]
        A_0 = gauss(A, b)[0]
        if np.isnan(solution[0]):
            print("There are an infinite number of solutions")
            print("Solution with free variable = 1")
            fv = 1
            free_variable_solution(A_0, b_0, fv, solution)
            print("Solution with free variable = -1")
            fv = -1
            free_variable_solution(A_0, b_0, fv, solution)
        elif np.isinf(solution[0]):
            print("No Solution")
        else:
            print(solution)

    def free_variable_solution(self, A_0, b_0, free_variable, solution_vector):
        x2 = free_variable
        x3 = solution_vector[len(solution_vector) - 1]
        x = sympy.symbols('x')
        expr = b_0[0] - A_0[0][0]*x - A_0[0][1]*x2  - A_0[0][2]*x3
        x1 = sympy.solve(expr)
        print(np.array([x1[0], x2, x3], float))

    def lu_decomp(self, A):
        (P, L, U) = sp.linalg.lu(A)
        print("L factor:")
        pprint.pprint(L)
        print("U factor:")
        pprint.pprint(U)
        return ""

A = np.array([
    [1,-1,2,-1],
    [2,-2,3,-3],
    [1,1,1,0],
    [1,-1,4,3]
    ],
    float)

b = np.array([-8,-20,-2,4], float)
data = Data(A, b)
print(data.check_valid_solution(A, b, data.gaussian, data.free_variable_solution))
print(data.lu_decomp(A))

**LDL^T**

In [ ]:
import numpy as np

def ldl_factorization(A):
    n = A.shape[0]
    L = np.eye(n)  # Initialize L as the identity matrix
    D = np.zeros((n, n))  # Initialize D as a zero matrix

    for k in range(n):
        # Update diagonal element of D
        D[k, k] = A[k, k] - np.sum(L[k, :k]**2 * D[:k, :k])

        # Update subdiagonal elements of L and superdiagonal elements of D
        for i in range(k+1, n):
            L[i, k] = (A[i, k] - np.sum(L[i, :k] * D[:k, k] * L[k, :k])) / D[k, k]

    return L, D

# Example usage:
A = np.array([[4, 12, -16],
              [12, 37, -43],
              [-16, -43, 98]])

L, D = ldl_factorization(A)

print("Matrix A:")
print(A)
print("\nLower triangular matrix L:")
print(L)
print("\nDiagonal matrix D:")
print(D)

# Verify the factorization
print("\nVerifying factorization (L * D * L^T):")
print(np.dot(np.dot(L, D), L.T))


**Permutation Matrix**

In [ ]:
import scipy.linalg as la
import numpy as np

def ldl_factorization(A):
    L, D, perm = la.ldl(A, lower=True, hermitian=True)
    return L, D, perm

# Example usage
A = np.array([[4, 12, -16], [12, 37, -43], [-16, -43, 98]])
L, D, perm = ldl_factorization(A)

print("L:")
print(L)
print("D:")
print(D)
print("Permutation:")
print(perm)

**Jacobi Iteration**

In [ ]:
import numpy as np
def JBI(A,b,x0,N,Tol):
    k = 1
    n = len(b)
    x = np.zeros(n)
    while k <= N:
        for i in range(n):
            xi = 1
            for j in range(n):
                if i != j:
                    xi -= A[i][j] * x0[j]
            x[i] = (xi + b[i]) / A[i][i]
        if np.max(np.abs(x - x0)) < Tol:
            return x
        k += 1
        x0 = x.copy()
    return "Maximum number of iterations exceeded."
A = np.array([
    [10, -1, 2, 0],
    [-1, 11, -1, 3],
    [2, -1, 10, -1],
    [0, 3, -1, 8]
])

b = np.array([6, 25, -11, 15])
Tol = 1.e-3
N = 10
x0 = np.array([0, 0, 0, 0])
print(JBI(A, b, x0, N, Tol))

[ 1.08774744  2.07833127 -0.89867614  1.10796333]


**Gauss_Seidel Iterative**

In [ ]:
import numpy as np
def GSI(A,x0,b,N,Tol):
  D = np.zeros_like(A)
  L = -np.tril(A)
  U = -np.triu(A)
  np.fill_diagonal(D, np.diag(A))
  np.fill_diagonal(U, 0)
  np.fill_diagonal(L, 0)
  k=1
  while k <= N:
    Cg=np.linalg.inv(D-L)@b
    Tg=np.linalg.inv(D-L)@U
    B=Tg@x0+Cg
    if np.max(np.abs(B-x0))<Tol:
       return B
    k+=1
    x0=B
  return "Maximum number of iterations exceeded."
A = np.array([
    [10, -1, 2, 0],
    [-1, 11, -1, 3],
    [2, -1, 10, -1],
    [0, 3, -1, 8]
])
b = np.array([6, 25, -11, 15])
Tol = 1.e-3
N = 5
x0 = np.array([0, 0, 0, 0])
print(GSI(A, x0, b, N, Tol))


[ 1.00009128  2.00002134 -1.00003115  0.9999881 ]


**SOR**

In [ ]:

import numpy as np
def SOR(A,x0,b,N,Tol,w):
  D = np.zeros_like(A)
  L = -np.tril(A)
  U = -np.triu(A)
  np.fill_diagonal(D, np.diag(A))
  np.fill_diagonal(U, 0)
  np.fill_diagonal(L, 0)
  k=1
  while k <= N:
    Cw=w*(np.linalg.inv(D-w*L))@b
    Tw=(np.linalg.inv(D-w*L))@((1-w)*D+w*U)
    B=Tw@x0+Cw
    if np.max(np.abs(B-x0))<Tol:
       return B
    k+=1
    x0=B
  return "Maximum number of iterations exceeded."
A = np.array([
    [4, 3, 0],
    [3, 4, -1],
    [0, -1, 4]
])
b = np.array([24, 30, -24])
Tol = 1.e-2
N = 7
w=1.25
x0 = np.array([0, 0, 0])
print(SOR(A,x0,b,N,Tol,w))